In [17]:
import numpy as np
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
# from IPython import display
from scipy.integrate import odeint

from IPython.display import display, clear_output, Image
# from plotly.widgets import GraphWidget

py.offline.init_notebook_mode(connected=True)

from plotly.offline import iplot

layout = go.Layout(
    title='',
    xaxis=dict(
        title="Time",
        range=[-0.1, 10],
        showline=True,
        mirror='ticks',
        zeroline=False
    ),
    yaxis=dict(
        title="$c/c_0$",
        range=[0,1],
        showline=True,
        mirror='ticks',
        zeroline=False
    )
)

times = np.linspace(0, 10, num=200)

# c0_wid = widgets.FloatSlider(min=0.1, max=10, value=1, description='c0')
# k0_wid = widgets.FloatSlider(min=0, max=10, value=0.3, description='k0')
# k1_wid = widgets.FloatSlider(min=0.1, max=50, value=1, description='k1')
# kTT_D_wid = widgets.FloatSlider(min=0, max=50, value=1, description='kTT_D')
# kTT_A_wid = widgets.FloatSlider(min=0, max=50, value=1, description='kTT_A')
# kTT_AD_wid = widgets.FloatSlider(min=0, max=50, value=1, description='kTT_AD')
# kdelay_wid = widgets.FloatSlider(min=0.1, max=10, value=1, description='kdelay')
# kd_wid = widgets.FloatSlider(min=0.1, max=4, value=0.2, description='kd')

# data = go.Scatter(x=[], y=[], mode='lines') 
# data = go.Scatter(x=xs, y=np.sin(xs/10 * 1), mode='lines')

fig = go.FigureWidget(data=[
    dict(
        type='scatter',
        mode='lines',
        line=dict(
                color='green',
                width=2
            )),
    dict(
        type='scatter',
        mode='lines',
        line=dict(
            color='black',
            width=0.5,
            dash='dash'
            )),
    dict(
        type='scatter',
        mode='lines',
        line=dict(
            color='black',
            width=0.5,
            dash='dot'
            )),
     dict(
        type='scatter',
        mode='lines',
        line=dict(
            color='red',
            width=2
            )),
    dict(
        type='scatter',
        mode='lines',
        line=dict(
            color='blue',
            width=3,
            dash='dot'
            ))
], layout=layout)


fig.data[0].x = times
fig.data[1].x = times
fig.data[2].x = times
fig.data[3].x = times
fig.data[4].x = times
fig.data[0].name = 'Xan'
fig.data[1].name = 'Ex'
fig.data[2].name = 'DR1'
fig.data[3].name = 'DR1+Ex'
fig.data[4].name = 'Delayed fluorescence (Xan * DR1)'

def Tall(A):
    result = (1 - np.power(10, -A)) / (A * np.log(10))
    result[np.isnan(result)] = 1
    return result
    

def update(k0, k1, kex, k_ex, kd, A0, kTT, kdel, x):
    
    def solve(conc, t):
        cXan, cEx, cDR, cDF = conc

        dcXan_dt = -(k0 + k1) * cXan # - kTT * cA*cC  # - kTT_D * cA ** 2 - kTT_AD * cA * cB
        dcEx_dt = + kex * cDR - k_ex * cEx  - kd * cEx
        dcDR_dt = + k1 * cXan + k_ex * cEx - kd * cDR - kex * cDR
        dcDF_dt = kTT * cXan*(x*cEx + (1-x)*cDR) - kdel * cDF

        return [dcXan_dt, dcEx_dt, dcDR_dt, dcDF_dt]
    
#     c0 = 2.3e-5
    
    result = odeint(solve, [1, 0, 0, 0], times)
    Xan = result[:, 0] 
    Ex = result[:, 1] 
    DR = result[:, 2]
    DF = result[:, 3]
    
    
    DF *= Tall(Xan * A0) if A0 > 0 else 1  # selfabsorption correction
    
    
    fig.data[0].y = Xan
    fig.data[1].y = Ex
    fig.data[2].y = DR
    fig.data[3].y = DR + Ex
    fig.data[4].y = DF / np.max(DF) * 0.3


# py.offline.iplot(fig)

# c0_wid = widgets.FloatSlider(min=0.1, max=10, value=1, description='c0')
k0_wid = widgets.FloatSlider(min=0.05, max=2, value=0.3, description='k<sub>0</sub>')
# k1_wid = widgets.FloatSlider(min=0.1, max=10, value=1, description='k<sub>1</sub>')
k1_wid = widgets.FloatLogSlider(
    value=1,
    base=10,
    min=-3, # max exponent of base
    max=3, # min exponent of base
    step=0.2, # exponent step
    description='k<sub>1</sub>'
)
# kex_wid = widgets.FloatSlider(min=0, max=50, value=1, description='k<sub>ex</sub>')

kex_wid = widgets.FloatLogSlider(
    value=1,
    base=10,
    min=-5, # max exponent of base
    max=5, # min exponent of base
    step=0.2, # exponent step
    description='k<sub>ex</sub>'
)
kTT_wid = widgets.FloatLogSlider(
    value=1,
    base=10,
    min=-1, # max exponent of base
    max=10, # min exponent of base
    step=0.2, # exponent step
    description='k<sub>TT</sub>'
)
k_ex_wid = widgets.FloatSlider(min=0, max=50, value=1, description='k<sub>-ex</sub>')
# kTT_AD_wid = widgets.FloatSlider(min=0, max=50, value=1, description='kTT_AD')
# kdelay_wid = widgets.FloatSlider(min=0.1, max=100, value=1, description='kdelay')
kd_wid = widgets.FloatSlider(min=0.1, max=4, value=0.2, description='k<sub>d</sub>')
A0_wid = widgets.FloatSlider(min=0, max=2, value=0, description='A<sub>0</sub>')
# kTT_wid = widgets.FloatSlider(min=0.1, max=1e8, value=100, description='k<sub>TT</sub>')
kdel_wid = widgets.FloatSlider(min=0.1, max=500, value=1000, description='k<sub>del</sub>')
kx_wid = widgets.FloatSlider(min=0, max=1, value=0.5, description='x')



# update(1, 20, 2, 1, 1)


## Differential equations

\begin{align}
\label{rce:vytezek}
\frac{\mathrm d[^3\mathrm{Xan}^*]}{\mathrm dt} &= - \left( k_0 + k_1\right) [^3\mathrm{Xan}^*]\\
\frac{\mathrm d[^3\mathrm{Ex}]}{\mathrm dt} &= k_1[^3\mathrm{Xan}^*] + k_{-ex}[^3\mathrm{DR1}] - k_{ex}[^3\mathrm{Ex}] - k_d[^3\mathrm{Ex}]\\
\frac{\mathrm d[^3\mathrm{DR1}]}{\mathrm dt} &= k_{ex}[^3\mathrm{Ex}] - k_{-ex}[^3\mathrm{DR1}] - k_d[^3\mathrm{DR1}]\\
\end{align}

$$I_{df}\propto \Phi_{\mathrm F, \mathrm{DR1}}[^3\mathrm{Xan}^*][^3\mathrm{DR1}]$$


In [20]:
widgets.interactive(update, k0=k0_wid, k1=k1_wid, kex=kex_wid, k_ex=k_ex_wid, kd=kd_wid, A0=A0_wid, kTT=kTT_wid, kdel=kdel_wid, x=kx_wid)
display(widgets.HBox([k0_wid, k1_wid]))
display(widgets.HBox([kex_wid, k_ex_wid]))
display(widgets.HBox([kd_wid]))
display(widgets.HBox([kTT_wid, kdel_wid]))
display(widgets.HBox([A0_wid]))
display(widgets.HBox([kx_wid]))

# iplot(fig)
display(fig)

FigureWidget({
    'data': [{'line': {'color': 'green', 'width': 2},
              'mode': 'lines',
          …